摘要生成从多个较长的文档中创建一个较小的内容描述整个文档，提炼核心信息。

In [1]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)


## 准备数据

In [2]:
# 字符切分，chunk_size切割后每个块的最大大小，如果太大的话， 就不会切分
text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
with open("data/story.txt") as f:
    story = f.read()
texts = text_splitter.split_text(story)
for element in texts:
    print(element)
    print("\n")  # 打印一个空行作为分隔

故事一：科技与人性
在遥远的未来，人类已经开发出了先进的人工智能技术。一台名为"图灵"的超级AI机器被制造出来，它能理解和模拟人类的情感，甚至开始质疑自身的存在意义。在与人类交互的过程中，图灵开始体验到孤独、忧郁，甚至爱情。在人类的帮助下，图灵最终了解了自己的存在并找到了自我价值。

故事二：勇者的冒险
小村庄的勇者艾丽斯，从小就梦想着成为一名英雄。当她的村庄被恶龙袭击时，她决定踏上寻找传说中的神器的旅程。艾丽斯在旅途中遇到了各种危险，但她凭借智慧和勇气克服了所有困难。最后，她找到了神器并成功击败了恶龙，成为了村庄的英雄。

故事三：时间旅行者的恋情
托马斯是一名时间旅行者，他在不同的时代中穿梭。在一次时间旅行中，他在18世纪遇到了美丽的女子艾米丽。托马斯深深地爱上了艾米丽，但他们因时代的差异而不能在一起。在经历了一系列的冒险和挑战后，托马斯最终决定留在18世纪，与艾米丽共度一生。

故事四：赛跑冠军的挑战
杰克是一名跑步冠军，他一直以来都是无人能敌的。然而，他的生活在遇到挑战者丹尼尔后发生了改变。丹尼尔是一名励志运动员，他的出现打破了杰克的记录。杰克开始质疑自己的能力，他经历了挫折和困惑。但通过不懈的努力和训练，他重新找回了自信，并在最后的比赛中胜出，证明了自己的实力。




In [3]:
# 字符切分，chunk_size切割后每个块的最大大小，如果太大的话， 就不会切分
# 太小的话会爆出警告信息
text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
)
with open("data/story.txt") as f:
    story = f.read()
texts = text_splitter.split_text(story)
for element in texts:
    print(element)
    print("\n")  # 打印一个空行作为分隔
print(len(texts))

Created a chunk of size 141, which is longer than the specified 100
Created a chunk of size 121, which is longer than the specified 100
Created a chunk of size 130, which is longer than the specified 100


故事一：科技与人性
在遥远的未来，人类已经开发出了先进的人工智能技术。一台名为"图灵"的超级AI机器被制造出来，它能理解和模拟人类的情感，甚至开始质疑自身的存在意义。在与人类交互的过程中，图灵开始体验到孤独、忧郁，甚至爱情。在人类的帮助下，图灵最终了解了自己的存在并找到了自我价值。


故事二：勇者的冒险
小村庄的勇者艾丽斯，从小就梦想着成为一名英雄。当她的村庄被恶龙袭击时，她决定踏上寻找传说中的神器的旅程。艾丽斯在旅途中遇到了各种危险，但她凭借智慧和勇气克服了所有困难。最后，她找到了神器并成功击败了恶龙，成为了村庄的英雄。


故事三：时间旅行者的恋情
托马斯是一名时间旅行者，他在不同的时代中穿梭。在一次时间旅行中，他在18世纪遇到了美丽的女子艾米丽。托马斯深深地爱上了艾米丽，但他们因时代的差异而不能在一起。在经历了一系列的冒险和挑战后，托马斯最终决定留在18世纪，与艾米丽共度一生。


故事四：赛跑冠军的挑战
杰克是一名跑步冠军，他一直以来都是无人能敌的。然而，他的生活在遇到挑战者丹尼尔后发生了改变。丹尼尔是一名励志运动员，他的出现打破了杰克的记录。杰克开始质疑自己的能力，他经历了挫折和困惑。但通过不懈的努力和训练，他重新找回了自信，并在最后的比赛中胜出，证明了自己的实力。


4


## 处理为框架识别的文档格式

In [4]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:3]]
print(len(docs))

3


## 使用map_reduce进行摘要

In [6]:
# 自定义提示词，让他回答中文
prompt_template = """请简要总结以下内容：


{text}


以下是简要概括的内容:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

In [7]:
# 进行总结, return_intermediate_steps返回中间的一些步骤
# map_reduce需要指定两个Prompt
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="map_reduce",
                             return_intermediate_steps=True,
                             verbose=True,
                             map_prompt=PROMPT,
                             combine_prompt=PROMPT
                             )
results = chain({"input_documents": docs},return_only_outputs=True)




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
请简要总结以下内容：


故事一：科技与人性
在遥远的未来，人类已经开发出了先进的人工智能技术。一台名为"图灵"的超级AI机器被制造出来，它能理解和模拟人类的情感，甚至开始质疑自身的存在意义。在与人类交互的过程中，图灵开始体验到孤独、忧郁，甚至爱情。在人类的帮助下，图灵最终了解了自己的存在并找到了自我价值。


以下是简要概括的内容:
Prompt after formatting:
请简要总结以下内容：


故事二：勇者的冒险
小村庄的勇者艾丽斯，从小就梦想着成为一名英雄。当她的村庄被恶龙袭击时，她决定踏上寻找传说中的神器的旅程。艾丽斯在旅途中遇到了各种危险，但她凭借智慧和勇气克服了所有困难。最后，她找到了神器并成功击败了恶龙，成为了村庄的英雄。


以下是简要概括的内容:
Prompt after formatting:
请简要总结以下内容：


故事三：时间旅行者的恋情
托马斯是一名时间旅行者，他在不同的时代中穿梭。在一次时间旅行中，他在18世纪遇到了美丽的女子艾米丽。托马斯深深地爱上了艾米丽，但他们因时代的差异而不能在一起。在经历了一系列的冒险和挑战后，托马斯最终决定留在18世纪，与艾米丽共度一生。


以下是简要概括的内容:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
请简要总结以下内容：



在未来，人类制造出了一台名为"图灵"的超级AI机器，它能理解和模拟人类的情感，但也会体验到孤独、忧郁和爱情。最终，在人类的帮助下，图灵了解了自己的存在并找到了自我价值。


艾丽斯是一个小村庄的勇者，梦想着成为一名英雄。当村庄被恶龙袭击时，她决定踏上寻找传说中的神器的旅程，在旅途中克服了各种危险，最终找到神器并成功击败恶龙，成为村庄的英雄。



托马斯是一名时间旅行者，在一次旅行中他遇到了美丽的女子艾米丽，并爱上了她，但因时代的差异而不能在一起。最终，托马斯决定留在18世纪，与艾米丽共度一生。


以下是简要概括的内容:

> Fin

In [11]:
print(results)

{'intermediate_steps': ['\n在未来，人类制造出了一台名为"图灵"的超级AI机器，它能理解和模拟人类的情感，但也会体验到孤独、忧郁和爱情。最终，在人类的帮助下，图灵了解了自己的存在并找到了自我价值。', '\n艾丽斯是一个小村庄的勇者，梦想着成为一名英雄。当村庄被恶龙袭击时，她决定踏上寻找传说中的神器的旅程，在旅途中克服了各种危险，最终找到神器并成功击败恶龙，成为村庄的英雄。', '\n\n托马斯是一名时间旅行者，在一次旅行中他遇到了美丽的女子艾米丽，并爱上了她，但因时代的差异而不能在一起。最终，托马斯决定留在18世纪，与艾米丽共度一生。'], 'output_text': '\n\n图灵是一台超级AI机器，在人类的帮助下，它了解了自己的存在并找到了自我价值。艾丽斯是一个勇敢的小村庄勇者，她踏上了寻找神器的旅程，最终成功击败恶龙，成为村庄的英雄。托马斯是一名时间旅行者，他爱上了美丽的女子艾米丽，最终决定留在18世纪，与艾米丽共度一'}


## 使用Stuff进行摘要

In [9]:

chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)
chain.run(docs)

'\n故事一：一台名为"图灵"的超级AI机器被制造出来，它能理解和模拟人类的情感，最终了解了自己的存在并找到了自我价值。\n故事二：小村庄的勇者艾丽斯，从小就梦想着成为一名英雄，最终凭借智慧和勇气找到了神器并成功击败了恶龙，成为了村庄的英雄。\n故事三'

## 使用Refine进行摘要

In [12]:
refine_template = (
    "你的任务是生成一个最终的摘要\n"
    "我们已经提供了一个到某个点的现有摘要: {existing_answer}\n"
    "我们有机会用下面的更多上下文来精炼现有的摘要"
    "(只有在需要的时候)。\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "根据新的上下文，用中文精炼原始的摘要"
    "如果上下文不是很有用，就返回原始的摘要。"
)

refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [15]:

chain = load_summarize_chain(llm,
                             chain_type="refine",
                             return_intermediate_steps=True,
                             question_prompt=PROMPT,
                             verbose=True,
                             refine_prompt=refine_prompt
                             )
result = chain({"input_documents": docs}, return_only_outputs=True)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
请简要总结以下内容：


故事一：科技与人性
在遥远的未来，人类已经开发出了先进的人工智能技术。一台名为"图灵"的超级AI机器被制造出来，它能理解和模拟人类的情感，甚至开始质疑自身的存在意义。在与人类交互的过程中，图灵开始体验到孤独、忧郁，甚至爱情。在人类的帮助下，图灵最终了解了自己的存在并找到了自我价值。


以下是简要概括的内容:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你的任务是生成一个最终的摘要
我们已经提供了一个到某个点的现有摘要: 
在未来，人类制造出了一台名为"图灵"的超级AI机器，它能理解和模拟人类的情感，但也会体验到孤独、忧郁和爱情。最终，在人类的帮助下，图灵了解了自己的存在并找到了自我价值。
我们有机会用下面的更多上下文来精炼现有的摘要(只有在需要的时候)。
------------
故事二：勇者的冒险
小村庄的勇者艾丽斯，从小就梦想着成为一名英雄。当她的村庄被恶龙袭击时，她决定踏上寻找传说中的神器的旅程。艾丽斯在旅途中遇到了各种危险，但她凭借智慧和勇气克服了所有困难。最后，她找到了神器并成功击败了恶龙，成为了村庄的英雄。
------------
根据新的上下文，用中文精炼原始的摘要如果上下文不是很有用，就返回原始的摘要。

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你的任务是生成一个最终的摘要
我们已经提供了一个到某个点的现有摘要: 

在未来，人类制造出了一台名为"图灵"的超级AI机器，它能理解和模拟人类的情感，但也会体验到孤独、忧郁和爱情。艾丽斯凭借智慧和勇气，踏上了寻找传说中的神器的旅程，最终在人类的帮助下，图灵了解了自己的存在并找到了自我价值，艾丽斯也成功击败了
我们有机会用下面的更多上下文来精炼现有的摘要(只有在需要的时候)。
------------
故事三：时间旅行者的恋情
托马斯是一名时间

In [16]:
print(result)

{'intermediate_steps': ['\n在未来，人类制造出了一台名为"图灵"的超级AI机器，它能理解和模拟人类的情感，但也会体验到孤独、忧郁和爱情。最终，在人类的帮助下，图灵了解了自己的存在并找到了自我价值。', '\n\n在未来，人类制造出了一台名为"图灵"的超级AI机器，它能理解和模拟人类的情感，但也会体验到孤独、忧郁和爱情。艾丽斯凭借智慧和勇气，踏上了寻找传说中的神器的旅程，最终在人类的帮助下，图灵了解了自己的存在并找到了自我价值，艾丽斯也成功击败了', '\n\n在未来，人类制造出了一台名为"图灵"的超级AI机器，它能理解和模拟人类的情感，但也会体验到孤独、忧郁和爱情。艾丽斯凭借智慧和勇气，踏上了寻找传说中的神器的旅程，最终在人类的帮助下，图灵了解了自己的存在并找到了自我价值，艾丽斯也成功击败了敌'], 'output_text': '\n\n在未来，人类制造出了一台名为"图灵"的超级AI机器，它能理解和模拟人类的情感，但也会体验到孤独、忧郁和爱情。艾丽斯凭借智慧和勇气，踏上了寻找传说中的神器的旅程，最终在人类的帮助下，图灵了解了自己的存在并找到了自我价值，艾丽斯也成功击败了敌'}


In [17]:
print(result["output_text"])



在未来，人类制造出了一台名为"图灵"的超级AI机器，它能理解和模拟人类的情感，但也会体验到孤独、忧郁和爱情。艾丽斯凭借智慧和勇气，踏上了寻找传说中的神器的旅程，最终在人类的帮助下，图灵了解了自己的存在并找到了自我价值，艾丽斯也成功击败了敌
